In [1]:
!wget http://cb.lk/covid_19

--2021-05-03 07:01:33--  http://cb.lk/covid_19
Resolving cb.lk (cb.lk)... 159.89.170.74
Connecting to cb.lk (cb.lk)|159.89.170.74|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cb.lk/covid_19 [following]
--2021-05-03 07:01:33--  https://cb.lk/covid_19
Connecting to cb.lk (cb.lk)|159.89.170.74|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/s/7rjw6oet4za01op/CovidDataset-20200427T133042Z-001.zip?dl=0 [following]
--2021-05-03 07:01:34--  https://www.dropbox.com/s/7rjw6oet4za01op/CovidDataset-20200427T133042Z-001.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/7rjw6oet4za01op/CovidDataset-20200427T133042Z-001.zip [following]
--2021-05-03 07:01:34--  https://www.dropbox.com/s/raw/7

In [ ]:
!unzip covid_19

Archive:  covid_19
replace CovidDataset/Val/Covid/88de9d8c39e946abd495b37cd07d89e5-6531-0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [2]:
TRAIN_PATH='CovidDataset/Train'
VAL_PATH='CovidDataset/Test'

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import *
from keras.preprocessing import image

In [4]:
model=Sequential()
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(224,224,3)))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))


In [5]:
model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

In [6]:
model.compile(loss=keras.losses.binary_crossentropy,optimizer='adam',metrics=['accuracy'])

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 110, 110, 64)      0         
_________________________________________________________________
dropout (Dropout)            (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 108, 108, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 54, 54, 64)        0

In [8]:
train_datagen=image.ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_dataset=image.ImageDataGenerator(rescale=1./255)

In [9]:
train_generator=train_datagen.flow_from_directory('CovidDataset/Train',target_size=(224,224),batch_size=32,class_mode='binary')
train_generator.class_indices

Found 224 images belonging to 2 classes.


{'Covid': 0, 'Normal': 1}

In [10]:
validation_generator=test_dataset.flow_from_directory('CovidDataset/Val',target_size=(224,224),batch_size=32,class_mode='binary')

Found 60 images belonging to 2 classes.


In [14]:
hist=model.fit(train_generator,epochs=10,validation_data=validation_generator)

Epoch 1/10
7/7 [==============================] - 75s 11s/step - loss: 0.6676 - accuracy: 0.6027 - val_loss: 0.6137 - val_accuracy: 0.6500
Epoch 2/10
7/7 [==============================] - 72s 10s/step - loss: 0.5236 - accuracy: 0.7143 - val_loss: 0.3938 - val_accuracy: 0.9167
Epoch 3/10
7/7 [==============================] - 74s 10s/step - loss: 0.4378 - accuracy: 0.7812 - val_loss: 0.3723 - val_accuracy: 0.8667
Epoch 4/10
7/7 [==============================] - 74s 11s/step - loss: 0.3128 - accuracy: 0.8750 - val_loss: 0.2704 - val_accuracy: 0.9667
Epoch 5/10
7/7 [==============================] - 75s 11s/step - loss: 0.2512 - accuracy: 0.9196 - val_loss: 0.1486 - val_accuracy: 0.9833
Epoch 6/10
7/7 [==============================] - 73s 10s/step - loss: 0.2406 - accuracy: 0.8884 - val_loss: 0.5718 - val_accuracy: 0.9333
Epoch 7/10
7/7 [==============================] - 73s 10s/step - loss: 0.2561 - accuracy: 0.9241 - val_loss: 0.1203 - val_accuracy: 0.9667
Epoch 8/10
7/7 [===========

In [15]:
model.evaluate_generator(train_generator)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


[0.11440090090036392, 0.96875]

In [16]:
model.evaluate_generator(validation_generator)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


[0.07686783373355865, 0.9833333492279053]